# Can Poisson distribution be used for betting?

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn
from scipy.stats import poisson,skellam

#https://www.football-data.co.uk/portugalm.php
league = pd.read_csv('P1.csv')
league = league[['HomeTeam','AwayTeam','FTHG','FTAG']].rename(columns={'FTHG': 'Home goals', 'FTAG': 'Away goals', 'AwayTeam': 'Away team', 'HomeTeam': 'Home team' })

In [71]:
league

,Home team,Away team,Home goals,Away goals
0,Benfica,Arouca,4,0
1,Rio Ave,Vizela,0,1
2,Estoril,Famalicao,2,0
3,Porto,Maritimo,5,1
4,Santa Clara,Casa Pia,0,0
...,...,...,...,...
76,Pacos Ferreira,Guimaraes,0,1
77,Boavista,Maritimo,1,1
78,Casa Pia,Vizela,0,1
79,Sp Braga,Chaves,0,1


In [52]:
#original_league = league
#league_till_last_round = league[-9:]
#league = league[:-9]
league.mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



Home goals    1.395062
Away goals    1.098765
dtype: float64

In [72]:
league_till_last_round = pd.read_excel('/content/Jornada_10_ligapt.xlsx')
league_till_last_round.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Home team   9 non-null      object 
 1   Away team   9 non-null      object 
 2   Home goals  0 non-null      float64
 3   Away goals  0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 416.0+ bytes


In [54]:
league[['Home goals','Away goals']].max()

Home goals    5
Away goals    6
dtype: int64

In [55]:
poisson.pmf(3,1.395062)*100

11.214025738951833

In [56]:
import plotly.graph_objects as go

#Y-Axis ticks for the line
max_goals = max(league[['Home goals','Away goals']].max())+1
goals = []
for i in range(max_goals):
  goals.append(i)
  goals[i] = str(goals[i])

#Poisson predictions as percentage
prob_poisson_home = []
prob_poisson_away = []
for i in range(max_goals):
    prob_poisson_home.append(poisson.pmf(i, league.mean().to_list()[0])*100)
    prob_poisson_away.append(poisson.pmf(i, league.mean().to_list()[1])*100)

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=league['Home goals'],
    histnorm='percent',
    name='Home goals',
    marker_color='#E13C3C',
    opacity=0.65
))
fig.add_trace(go.Histogram(
    x=league['Away goals'],
    histnorm='percent',
    name='Away goals',
    marker_color='#4C3CE1',
    opacity=0.65
))

fig.add_trace(go.Scatter(x=goals, y=prob_poisson_home,
                    mode='lines+markers',
                    line = dict(color='#E13C3C', width=4, dash='dash'),
                    name='Home goals - Poisson', ))

fig.add_trace(go.Scatter(x=goals, y=prob_poisson_away,
                    mode='lines+markers',
                    line = dict(color='#4C3CE1', width=4, dash='dash'),
                    name='Away goals - Poisson'))

fig.update_layout(title='Real nº goals per match vs Poisson prediction\n         Liga Portugal 2021/22',
                    title_font_family="Calibri",
                    title_font_color="black",
                   xaxis_title='Goals in a match',
                   yaxis_title='% Probability',
                    barmode='group')

fig.update_layout({
'plot_bgcolor': 'rgba(255,250,250 250)',
'paper_bgcolor': 'rgba(255,250,250,250)',
})



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



## Predict a match of the Premier League

In [57]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model_data = pd.concat([league[['Home team','Away team','Home goals']].assign(home=1).rename(
            columns={'Home team':'team', 'Away team':'opponent','Home goals':'goals'}),
           league[['Away team','Home team','Away goals']].assign(home=0).rename(
            columns={'Away team':'team', 'Home team':'opponent','Away goals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                  162
Model:                            GLM   Df Residuals:                      126
Model Family:                 Poisson   Df Model:                           35
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -198.56
Date:                Mon, 17 Oct 2022   Deviance:                       133.13
Time:                        20:50:46   Pearson chi2:                     116.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.6546      0.419      1.561      0.119      -0.168       1.477
team[T.Benfica]                0.4853      0.383      1.268      0.205      -0.265       1.236
team[T.Boavista]              -0.4116      0.462     -0.892      0.373      -1.316       0.493
team[T.Casa Pia]              -0.4002      0.459     -0.872      0.383      -1.299       0.499
team[T.Chaves]                -0.1723      0.491     -0.351      0.726      -1.134       0.790
team[T.Estoril]                0.0869      0.442      0.197      0.844      -0.780       0.954
team[T.Famalicao]             -0.5499      0.517     -1.064      0.287      -1.563       0.463
team[T.Gil Vicente]           -0.4860      0.482     -1.008      0.313      -1.431       0.459
team[T.Guimaraes]             -0.3883      0.496     -0.783      0.434      -1.361       0.584
team[T.Maritimo]              -0.3772      0.518     -0.728      0.467      -1.393       0.639
team[T.Pacos Ferreira]        -0.5379      0.519     -1.037      0.300      -1.555       0.479
team[T.Portimonense]          -0.3606      0.477     -0.756      0.450      -1.296       0.574
team[T.Porto]                  0.7386      0.398      1.856      0.063      -0.041       1.519
team[T.Rio Ave]                0.4745      0.431      1.102      0.271      -0.370       1.319
team[T.Santa Clara]           -0.8756      0.519     -1.687      0.092      -1.893       0.142
team[T.Sp Braga]               0.7460      0.393      1.897      0.058      -0.025       1.517
team[T.Sp Lisbon]              0.6286      0.401      1.567      0.117      -0.158       1.415
team[T.Vizela]                -0.1356      0.503     -0.269      0.788      -1.122       0.851
opponent[T.Benfica]           -1.1299      0.527     -2.144      0.032      -2.163      -0.097
opponent[T.Boavista]          -0.2793      0.387     -0.722      0.470      -1.038       0.479
opponent[T.Casa Pia]          -1.0923      0.521     -2.097      0.036      -2.113      -0.071
opponent[T.Chaves]            -1.0777      0.437     -2.464      0.014      -1.935      -0.220
opponent[T.Estoril]           -1.0810      0.472     -2.288      0.022      -2.007      -0.155
opponent[T.Famalicao]         -0.4717      0.388     -1.216      0.224      -1.232       0.288
opponent[T.Gil Vicente]       -0.6201      0.410     -1.513      0.130      -1.423       0.183
opponent[T.Guimaraes]         -1.2102      0.485     -2.496      0.013      -2.160      -0.260
opponent[T.Maritimo]           0.1125      0.331      0.340      0.734      -0.536       0.761
opponent[T.Pacos Ferreira]     0.0553      0.356      0.155      0.877      -0.642       0.753
opponent[T.Portimonense]      -0.7564      0.440     -1.720      0.085      -1.618       0.105
opponent[T.Porto]             -1.3245      

In [28]:
import math
benfica_home_goals = math.exp(0.0436+0.8868-0.5195+0.1496)
porto_away_goals = math.exp(0.0436+0.9801-0.8177)

display(benfica_home_goals, porto_away_goals)

1.7515480554167886

1.2287532039953122

## Testing the last matchday

In [66]:
league_till_last_round = league_till_last_round.reset_index()

In [73]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))


In [74]:
from numpy import unravel_index

home_match = []
away_match = []
home_goals = []
away_goals = []
pred_list_home = []
pred_list_away = []

for n in range(0,len(league_till_last_round['Home team'])):

    a = simulate_match(poisson_model, league_till_last_round['Home team'].loc[n], league_till_last_round['Away team'].loc[n], max_goals=7)
    b = unravel_index(a.argmax(), a.shape)
    
    home_match.append(league_till_last_round['Home team'].loc[n])
    away_match.append(league_till_last_round['Away team'].loc[n])
    home_goals.append(league_till_last_round['Home goals'].loc[n])
    away_goals.append(league_till_last_round['Away goals'].loc[n])
    pred_list_home.append(b[0])
    pred_list_away.append(b[1])
    

final_df = pd.DataFrame({'Home': home_match, 'Away': away_match,
                         'Home Goals': home_goals, 'Away Goals': away_goals,
                         'Home Prediction': pred_list_home, 'Away Prediction': pred_list_away})

In [75]:
league_till_last_round

,Home team,Away team,Home goals,Away goals
0,Porto,Benfica,NaN,NaN
1,Famalicao,Pacos Ferreira,NaN,NaN
2,Estoril,Sp Braga,NaN,NaN
3,Sp Lisbon,Casa Pia,NaN,NaN
4,Vizela,Santa Clara,NaN,NaN
5,Maritimo,Arouca,NaN,NaN
6,Chaves,Gil Vicente,NaN,NaN
7,Guimaraes,Boavista,NaN,NaN
8,Rio Ave,Portimonense,NaN,NaN


In [76]:
def transform_df(mod_df):
    mod_df.loc[(mod_df['Home Goals'] > mod_df['Away Goals']), '1x2'] = '1'
    mod_df.loc[(mod_df['Home Goals'] == mod_df['Away Goals']), '1x2'] = 'X'
    mod_df.loc[(mod_df['Home Goals'] < mod_df['Away Goals']), '1x2'] = '2'
    
    mod_df['Home Prediction Int'] = mod_df['Home Prediction'].round(0).astype(int)
    mod_df['Away Prediction Int'] = mod_df['Away Prediction'].round(0).astype(int)
    mod_df.loc[(mod_df['Home Prediction Int'] > mod_df['Away Prediction Int']), '1x2_Pred'] = '1'
    mod_df.loc[(mod_df['Home Prediction Int'] == mod_df['Away Prediction Int']), '1x2_Pred'] = 'X'
    mod_df.loc[(mod_df['Home Prediction Int'] < mod_df['Away Prediction Int']), '1x2_Pred'] = '2'
    
    mod_df['Exact_Res'] = 'NO'
    mod_df.loc[(mod_df['Home Goals'] == mod_df['Home Prediction Int']) & 
               (mod_df['Away Goals'] == mod_df['Away Prediction Int']), 'Exact_Res'] = 'YES'
    
    mod_df.loc[mod_df['1x2'] == mod_df['1x2_Pred'], 'Match'] = 'Yes'
    mod_df.loc[mod_df['1x2'] != mod_df['1x2_Pred'], 'Match'] = 'No'
    return mod_df

In [77]:
transform_df(final_df)[['Home','Away','Home Prediction','Away Prediction','1x2_Pred']]

,Home,Away,Home Prediction,Away Prediction,1x2_Pred
0,Porto,Benfica,1,0,1
1,Famalicao,Pacos Ferreira,1,0,1
2,Estoril,Sp Braga,1,1,X
3,Sp Lisbon,Casa Pia,1,0,1
4,Vizela,Santa Clara,1,0,1
5,Maritimo,Arouca,1,2,2
6,Chaves,Gil Vicente,1,0,1
7,Guimaraes,Boavista,1,0,1
8,Rio Ave,Portimonense,1,0,1


In [ ]:
matching_results = final_df.groupby('Match')['Match'].count()
# accuracy
accuracy = matching_results[0]*100/(matching_results[0] + matching_results[1])
accuracy_exact_result = len(final_df[final_df['Exact_Res'] == 'YES'])*100 / len(final_df)

print(f'Winning game accuracy: {accuracy}% \nExact result accuracy: {accuracy_exact_result}%')


Winning game accuracy: 55.55555555555556% 
Exact result accuracy: 22.22222222222222%
